## ray training with pytorch

In [1]:
# !pip install 'ray[default]'

In [2]:
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms

import ray
from ray.util.sgd.torch import TorchTrainer
from ray.util.sgd.torch import TrainingOperator
# https://github.com/kuangliu/pytorch-cifar/blob/master/models/resnet.py
from ray.util.sgd.torch.resnet import ResNet18


In [4]:
ray.init("ray:// 10.3.249.181:10001")

In [3]:
def cifar_creator(config):
    """Returns dataloaders to be used in `train` and `validate`."""
    tfms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2023, 0.1994, 0.2010)),
    ])  # meanstd transformation
    train_loader = DataLoader(
        CIFAR10(root="~/data", download=True, transform=tfms), batch_size=config["batch"])
    validation_loader = DataLoader(
        CIFAR10(root="~/data", download=True, transform=tfms), batch_size=config["batch"])
    return train_loader, validation_loader

def optimizer_creator(model, config):
    """Returns an optimizer (or multiple)"""
    return torch.optim.SGD(model.parameters(), lr=config["lr"])

CustomTrainingOperator = TrainingOperator.from_creators(
    model_creator=ResNet18, # A function that returns a nn.Module
    optimizer_creator=optimizer_creator, # A function that returns an optimizer
    data_creator=cifar_creator, # A function that returns dataloaders
    loss_creator=torch.nn.CrossEntropyLoss  # A loss function
    )


In [5]:
trainer = TorchTrainer(
    training_operator_cls=CustomTrainingOperator,
    config={"lr": 0.01, # used in optimizer_creator
            "batch": 64 # used in data_creator
           },
    num_workers=2,  # amount of parallelism
    use_gpu=torch.cuda.is_available(),
    use_tqdm=True)

(pid=1925) 2021-08-26 20:07:53,474	INFO distributed_torch_runner.py:58 -- Setting up process group for: tcp://10.1.229.3:60591 [rank=0]
(pid=1930) 2021-08-26 20:07:53,509	INFO distributed_torch_runner.py:58 -- Setting up process group for: tcp://10.1.229.3:60591 [rank=1]


(pid=1925) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/data/cifar-10-python.tar.gz


 94%|█████████▍| 160483328/170498071 [00:02<00:00, 110651479.93it/s]
170499072it [00:02, 77378462.34it/s]                                


(pid=1925) Extracting /root/data/cifar-10-python.tar.gz to /root/data
(pid=1925) Files already downloaded and verified
(pid=1930) Files already downloaded and verified
(pid=1930) Files already downloaded and verified


In [ ]:
stats = trainer.train()
print(trainer.validate())

In [ ]:
torch.save(trainer.state_dict(), "checkpoint.pt")
trainer.shutdown()
print("success!")